Primeiro, baixe as bibliotecas do python em um ambiente virtual: 

python -m venv .venv

cd .venv\Scripts\activate

pip install pandas

pip install numpy

pip install fastparquet

pip install scikit-learn

pip install matplotlib

pip install seaborn


In [11]:
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
from sqlalchemy import text


# 🔴 SUBSTITUA pelos seus dados reais
DB_USER = "cryslayne"
DB_PASSWORD = "crys0231"
DB_HOST = "db-data-dadosfera.c9e0k2cgao73.sa-east-1.rds.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "postgres"

DATABASE_URL = (
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}"
    f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

try:
    engine = create_engine(DATABASE_URL)
    with engine.connect() as conn:
        result = conn.execute(text("SELECT current_database(), version();"))
        print("✅ Conexão bem-sucedida!")
        print(result.fetchone())
except OperationalError as e:
    print("❌ Erro de conexão:")
    print(e)


✅ Conexão bem-sucedida!
('postgres', 'PostgreSQL 17.6 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit')


In [1]:
import pandas as pd
import numpy as np
import random

from datetime import datetime, timedelta

pd.set_option("display.max_columns", None)
random.seed(42)
np.random.seed(42)


In [14]:
# Generate products_raw.json

N_PRODUCTS = 10_000

categories = ["Rolamentos", "Mancais", "Componentes Mecânicos"]
bearing_types = ["Esférico", "Cilíndrico", "Autocompensador", "Agujas", "Contato Angular"]
materials = ["Aço", "Aço Inoxidável", "Cerâmica"]
problem_types = ["Vibração", "Superaquecimento", "Desgaste", "Contaminação"]

products = []

for i in range(N_PRODUCTS):
    products.append({
        "product_id": f"P{i+1:05d}",
        "product_name": f"Rolamento Industrial {i+1}",
        "product_category": random.choice(categories),
        "product_subcategory": random.choice(bearing_types),
        "manufacturer": random.choice(["SKF", "NSK", "FAG", "Timken", "NTN"]),
        "model": f"MD-{random.randint(100,999)}",
        "bearing_type": random.choice(bearing_types),
        "material": random.choice(materials),
        "load_capacity": round(random.uniform(500, 50000), 2),
        "max_speed": random.randint(1000, 15000),
        "temperature_limit": random.randint(80, 250),
        "problem_type": random.choice(problem_types),
        "unit_cost": round(random.uniform(50, 500), 2),
        "list_price": round(random.uniform(200, 3000), 2)
    })


products_df = pd.DataFrame(products)
products_df.to_json("../data/raw/products_raw.json", orient="records", indent=2)

products_df.head()


,product_id,product_name,product_category,product_subcategory,manufacturer,model,bearing_type,material,load_capacity,max_speed,temperature_limit,problem_type,unit_cost,list_price
0,P00001,Rolamento Industrial 1,Componentes Mecânicos,Esférico,SKF,MD-859,Autocompensador,Aço,11548.93,13066,106,Vibração,315.72,288.99
1,P00002,Rolamento Industrial 2,Rolamentos,Cilíndrico,NSK,MD-617,Contato Angular,Aço,28281.63,12731,246,Contaminação,149.20,1849.94
2,P00003,Rolamento Industrial 3,Rolamentos,Cilíndrico,Timken,MD-448,Autocompensador,Aço,11158.03,13509,166,Vibração,91.74,470.81
3,P00004,Rolamento Industrial 4,Mancais,Contato Angular,FAG,MD-926,Esférico,Cerâmica,23241.53,3045,176,Vibração,298.42,2522.33
4,P00005,Rolamento Industrial 5,Componentes Mecânicos,Autocompensador,NTN,MD-296,Esférico,Aço,33232.53,13665,154,Vibração,434.89,2626.15


In [15]:
# Generate customers_raw.csv

N_CUSTOMERS = 5_000

industries = [
    "Mineração", "Siderurgia", "Alimentos", "Automotiva",
    "Papel e Celulose", "Química", "Cimento", "Energia"
]

customers = []

for i in range(N_CUSTOMERS):
    customers.append({
        "customer_id": f"C{i+1:05d}",
        "company_name": f"Empresa Industrial {i+1}",
        "industry": random.choice(industries),
        "company_size": random.choice(["Pequena", "Média", "Grande"]),
        "maintenance_model": random.choice(["Interna", "Terceirizada", "Mista"]),
        "equipment_criticality": random.choice(["Baixa", "Média", "Alta"]),
        "annual_revenue_estimated": round(random.uniform(5e6, 5e9), 2),
        "maintenance_budget_annual": round(random.uniform(50_000, 5_000_000), 2),
        "downtime_cost_per_hour": round(random.uniform(1_000, 50_000), 2),
        "preferred_supplier": random.choice([True, False]),
        "relationship_start_date": (
            datetime(2000,1,1) + timedelta(days=random.randint(0,9000))
        ).date(),
        "active": random.choice([True, True, True, False]),
        "last_updated": datetime.now()
    })

customers_df = pd.DataFrame(customers)
customers_df.to_csv("../data/raw/customers_raw.csv", index=False)

customers_df.head()


,customer_id,company_name,industry,company_size,maintenance_model,equipment_criticality,annual_revenue_estimated,maintenance_budget_annual,downtime_cost_per_hour,preferred_supplier,relationship_start_date,active,last_updated
0,C00001,Empresa Industrial 1,Química,Grande,Terceirizada,Baixa,1.474241e+09,2824887.23,3170.40,False,2016-10-22,True,2026-01-03 22:01:00.012768
1,C00002,Empresa Industrial 2,Automotiva,Grande,Mista,Média,3.453635e+09,2913305.71,41838.33,True,2022-03-26,True,2026-01-03 22:01:00.012974
2,C00003,Empresa Industrial 3,Alimentos,Pequena,Mista,Alta,4.505871e+09,154697.84,12088.05,True,2022-10-12,True,2026-01-03 22:01:00.013015
3,C00004,Empresa Industrial 4,Automotiva,Pequena,Mista,Alta,1.170286e+09,663312.06,10768.29,False,2001-10-06,True,2026-01-03 22:01:00.013035
4,C00005,Empresa Industrial 5,Automotiva,Pequena,Interna,Média,3.405395e+09,1099120.20,39752.67,True,2013-04-30,True,2026-01-03 22:01:00.013052


In [16]:
# Generate sales_raw.csv

N_SALES = 120_000
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 31)

def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

sales = []

for i in range(N_SALES):
    product = products_df.sample(1).iloc[0]
    customer = customers_df.sample(1).iloc[0]

    quantity = random.randint(1, 40)
    unit_price = round(product["list_price"] * random.uniform(0.9, 1.05), 2)

    sales.append({
        "sale_id": f"S{i+1:07d}",
        "sale_date": random_date(START_DATE, END_DATE).date(),
        "customer_id": customer["customer_id"],
        "product_id": product["product_id"],
        "quantity": quantity,
        "unit_price": unit_price,
        "total_price": round(quantity * unit_price, 2),
        "discount_percentage": random.choice([0, 5, 10, 15]),
        "sales_channel": random.choice(["Direct", "Distributor", "Representative"]),
        "contract_type": random.choice(["Spot", "Recurring", "SLA"]),
        "payment_terms": random.choice(["30 days", "60 days", "90 days"]),
        "delivery_lead_time_days": random.randint(3, 25),
        "sale_status": random.choice(["Completed", "Completed", "Completed", "Cancelled"]),
        "last_updated": datetime.now()
    })

sales_df = pd.DataFrame(sales)
sales_df.to_csv("../data/raw/sales_raw.csv", index=False)
sales_df.head()


,sale_id,sale_date,customer_id,product_id,quantity,unit_price,total_price,discount_percentage,sales_channel,contract_type,payment_terms,delivery_lead_time_days,sale_status,last_updated
0,S0000001,2023-11-02,C00100,P06253,9,455.17,4096.53,15,Distributor,Spot,60 days,25,Completed,2026-01-03 22:01:00.948305
1,S0000002,2025-05-01,C00983,P06975,5,1192.17,5960.85,0,Representative,SLA,90 days,16,Completed,2026-01-03 22:01:00.951185
2,S0000003,2023-11-12,C03975,P02021,26,553.04,14379.04,5,Distributor,Spot,30 days,17,Cancelled,2026-01-03 22:01:00.954457
3,S0000004,2025-05-09,C04737,P02529,5,2780.14,13900.70,10,Distributor,Recurring,90 days,20,Cancelled,2026-01-03 22:01:00.956871
4,S0000005,2023-05-20,C01652,P05798,20,1397.75,27955.00,15,Distributor,Recurring,30 days,7,Completed,2026-01-03 22:01:00.959928


In [19]:
# Validate referential integrity

print("Produtos:", products_df.shape)
print("Clientes:", customers_df.shape)
print("Vendas:", sales_df.shape)

assert sales_df["product_id"].isin(products_df["product_id"]).all()
assert sales_df["customer_id"].isin(customers_df["customer_id"]).all()

print("Integridade referencial validada")


Produtos: (10000, 14)
Clientes: (5000, 13)
Vendas: (120000, 14)
Integridade referencial validada


In [ ]:
from sqlalchemy import create_engine, text
import json
import os

print("="*80)
print("CARREGANDO DADOS NO POSTGRESQL RDS")
print("="*80)

# ===== MUDE AQUI =====
DB_USER = "cryslayne"
DB_PASSWORD = "crys0231"
DB_HOST = "db-data-dadosfera.c9e0k2cgao73.sa-east-1.rds.amazonaws.com"
DB_PORT = "5432"
DB_NAME = "postgres"


PRODUCTS_FILE = '../data/raw/products_raw.json'
CUSTOMERS_FILE = '../data/raw/customers_raw.csv'
SALES_FILE = '../data/raw/sales_raw.csv'



# ===== VERIFICAR ARQUIVOS =====
print("\nVerificando arquivos...")
for f in [PRODUCTS_FILE, CUSTOMERS_FILE, SALES_FILE]:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024 / 1024  # MB
        print(f"{f} ({size:.2f} MB)")
    else:
        print(f"{f} NÃO ENCONTRADO!")
        
# ===== CONECTAR AO POSTGRESQL =====
print("\nConectando ao PostgreSQL RDS...")

try:
    engine = create_engine(
        f"postgresql://{DB_USER}:{DB_PASSWORD}@"
        f"{DB_HOST}:{DB_PORT}/{DB_NAME}"
    )
    
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    
    print(f"Conectado ao RDS")
except Exception as e:
    print(f"Erro: {e}")
    exit()

# ===== CARREGAR ARQUIVOS =====
print("\nCarregando dados...")

try:
    with open(PRODUCTS_FILE, 'r') as f:
        products_df = pd.DataFrame(json.load(f))
    print(f"Products: {len(products_df):,} linhas")
    
    customers_df = pd.read_csv(CUSTOMERS_FILE)
    print(f"Customers: {len(customers_df):,} linhas")
    
    sales_df = pd.read_csv(SALES_FILE)
    print(f"Sales: {len(sales_df):,} linhas")
    
except Exception as e:
    print(f"Erro ao carregar: {e}")
    exit()

# ===== INSERIR NO BANCO =====
print("\nInserindo no PostgreSQL...")

try:
    with engine.connect() as conn:
        conn.execute(text("DROP TABLE IF EXISTS sales_raw CASCADE;"))
        conn.execute(text("DROP TABLE IF EXISTS products_raw CASCADE;"))
        conn.execute(text("DROP TABLE IF EXISTS customers_raw CASCADE;"))
        conn.commit()
    
    products_df.to_sql('products_raw', engine, if_exists='replace', index=False)
    print("products_raw inserida")
    
    customers_df.to_sql('customers_raw', engine, if_exists='replace', index=False)
    print("customers_raw inserida")
    
    sales_df.to_sql('sales_raw', engine, if_exists='replace', index=False)
    print("sales_raw inserida")
    
except Exception as e:
    print(f"Erro: {e}")
    exit()

# ===== CRIAR ÍNDICES =====
print("\nCriando índices...")

try:
    with engine.connect() as conn:
        conn.execute(text("CREATE INDEX idx_products_id ON products_raw(product_id);"))
        conn.execute(text("CREATE INDEX idx_customers_id ON customers_raw(customer_id);"))
        conn.execute(text("CREATE INDEX idx_sales_id ON sales_raw(sale_id);"))
        conn.execute(text("CREATE INDEX idx_sales_customer ON sales_raw(customer_id);"))
        conn.execute(text("CREATE INDEX idx_sales_product ON sales_raw(product_id);"))
        conn.commit()
    
    print("5 índices criados")
except:
    print("Índices já existem")

# ===== VERIFICAR =====
print("\nDados no PostgreSQL:")

with engine.connect() as conn:
    r1 = conn.execute(text("SELECT COUNT(*) FROM products_raw;")).fetchone()[0]
    r2 = conn.execute(text("SELECT COUNT(*) FROM customers_raw;")).fetchone()[0]
    r3 = conn.execute(text("SELECT COUNT(*) FROM sales_raw;")).fetchone()[0]

print(f"   • products_raw: {r1:,} registros")
print(f"   • customers_raw: {r2:,} registros")
print(f"   • sales_raw: {r3:,} registros")

print("\n" + "="*80)
print("CARREGAMENTO CONCLUÍDO!")
print("="*80)


CARREGANDO DADOS NO POSTGRESQL RDS

Verificando arquivos...
../data/raw/products_raw.json (4.34 MB)
../data/raw/customers_raw.csv (0.70 MB)
../data/raw/sales_raw.csv (13.93 MB)

🔌 Conectando ao PostgreSQL RDS...
Conectado ao RDS

Carregando dados...
Products: 10,000 linhas
Customers: 5,000 linhas
Sales: 120,000 linhas

Inserindo no PostgreSQL...
products_raw inserida
customers_raw inserida


In [20]:
# Structure

# data/
#  └── raw/
#      ├── products_raw.json   ✔ 10.000
#      ├── customers_raw.csv   ✔ 5.000
#      └── sales_raw.csv       ✔ 120.000